# Ensemble 
- 여러개의 분류기(Classifier)를 생성하고 예측을 결합하므로써 정확한 최종 예측 도출

## ✔ Ensemble 학습의 유형
- Voting
    - 여러개의 분류기(Classifier)가 투표를 통해 최종 예측 결과 결정
    - 서로 다른 알고리즘을 가진 분류기(Classifier)를 결합하므로써
- Bagging
    - 여러개의 분류기(Classifier)가 투표를 통해 최종 예측 결과 결정
    - 각각의 분류기(Classifier)가 같은 유형의 알고리즘이지만 data sampling을 서로 다르게 학습해 voting수행
    - Random Forest
- Boosting
    - 여러개의 분류기가 순차적으로 학습 수행 
    - 앞에서 학습한 분류기가 예측이 틀린 데이터에 대해 올바르게 예측할 수 있도록 다음 분류기에서 가중치를 부여하면서 학습과 예측 진행
    - Gradient Boost, XGBoost, LightGBM
- Stacking
    - 여러개의 다른 모델의 예측 결과값을 다시 학습 데이터로 만들어서 다른 모델로 재학습시켜 결과 예측


## 1. Voting
### (1) Hard Voting
- 예측한 결괏값들 중 다수의 분류기가 결정한 예측값을 최종 Voting 결과값으로 선정

### (2) Soft Voting
- 분류기들의 label값 결정 확률을 모두 더하고 평균내서 확률이 가장 높은 label값을 최종 Voting 결과값으로 선정
- 일반적으로 사용

## ✔ Voting 분류기 ( Voting Classifier )
- Voting방식의 Ensemble 이용해 위스콘신 유방암 Dataset 예측
- Logistic Regression & KNN 기반으로 Voting Classifier 생성

In [2]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
cancer=load_breast_cancer()

In [4]:
df=pd.DataFrame(cancer.data,columns=cancer.feature_names)

In [5]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### voting 분류기 만들기

In [6]:
# 개별 model
lr_clf=LogisticRegression()
knn_clf=KNeighborsClassifier(n_neighbors=8)

In [7]:
# 개별 model을 soft Voting기반의 Ensemble Model로 구현
vo_clf=VotingClassifier(estimators=[('LR',lr_clf),('KNN',knn_clf)], voting='soft')

In [8]:
X_train,X_test, y_train, y_test=train_test_split(cancer.data, cancer.target, test_size=0.2, random_state=156)

In [9]:
# Voting classifier 학습/ 예측 / 평가
vo_clf.fit(X_train, y_train)
pred=vo_clf.predict(X_test)

C:\Users\kimjh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
accuracy_score(y_test, pred)

0.9473684210526315

In [11]:
# 개별 model의 학습 / 예측 평가
classifier=[lr_clf, knn_clf]
for cl in classifier:
    cl.fit(X_train, y_train)
    pred=cl.predict(X_test)
    class_name=cl.__class__.__name__
    print(class_name, "의 정확도", accuracy_score(y_test, pred))

LogisticRegression 의 정확도 0.9385964912280702
KNeighborsClassifier 의 정확도 0.9385964912280702
C:\Users\kimjh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Result
- Voting Classifier의 정확도가 더 높게 나타나긴 했지만 항상 그런것은 아님

## 2. Bagging
- 같은 알고리즘으로 여러 개의 분류기를 만들어 Voting으로 최종 결정
- Ensemble Algorithm 중 가장 빠른 수행속도 가짐 
- 여러개의 Decision Tree 분류기가 전체 data에서 bagging 방식으로 각자의 data를 sampling 해 개별적으로 학습 수행

## ✔ Bootstrapping 
- 여러개의 dataset을 중첩되게 분리 

## ✔ Random Forest의 Hyper Parameter & Tuning
- 트리 기반 Ensemble Algorithm의 단점 
    - Hyper Parameter가 너무 많아서 Tuning을 위한 시간이 많이 소요
    - 시간을 많이 소요했음에도 Tuning 후 예측 성능이 크게 향상 X
- n_estimators 
    - Random Forest에서 Decision Tree의 갯수 지정
    - default => 10 개
    - 갯수를 늘릴수록 학습 수행 시간이 오래 걸림
- max_features
    - Random Forest의 Tree를 분할하는 feature를 참조할 때 sqrt(전체 feature 갯수)만큼 참조

## 3. Boosting
- 여러개의 약한 학습기를 순차적으로 학습 & 예측하면서 잘못한 데이터에 가중치를 부여해 오류를 개선하면서 학습

## ✔ AdaBoost
    - 오류 data에 가중치 부여하면서 Boosting 수행

## ✔ GBM (Gradient Boosting Machine)
    - AdaBoost와 유사하나 경사 하강법을 이용해 가중치 update
    - 과적합에도 강한 뛰어난 예측 성능 가짐
    - 수행시간 오래걸린다는 단점 

### 경사 하강법 (Gradient Descent)
- 오차를 최소화하는 방향성을 가지고 반복적으로 가중치 값을 update
- 회귀 문제, 분류문제 둘 다 해결 가능


## ✔ XGBoost (eXtra Gradient Boost)
- 분류에 있어서 다른 ML보다 뛰어남
- GBM에 기반
    - GBM의 단점인 느린 수행 시간 & 과적합 규제 부재(Regularization) 등의 문제를 해결 
- 병렬 CPU 환경에서 병렬 학습 가능  
- Tree Pruning 
    - 더 이상 긍정 이득이 없는 분할을 가지치기